In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
from src.inference import get_feature_store

In [6]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
current_date = current_date.replace(year=2024)
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-05-10 16:44:52,607 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 16:44:52,630 INFO: Initializing external client
2025-05-10 16:44:52,630 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-10 16:44:53,559 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214680
Fetching data from 2024-04-11 20:44:52.607303+00:00 to 2024-05-10 19:44:52.607303+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (11.68s) 


In [7]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2024-04-11 21:00:00+00:00,190 Morgan,0
1,2024-04-11 22:00:00+00:00,190 Morgan,0
2,2024-04-11 23:00:00+00:00,190 Morgan,0
3,2024-04-12 00:00:00+00:00,190 Morgan,0
4,2024-04-12 01:00:00+00:00,190 Morgan,0
...,...,...,...
395685,2024-05-10 04:00:00+00:00,4757.06,0
395686,2024-05-10 05:00:00+00:00,4757.06,0
395687,2024-05-10 09:00:00+00:00,4757.06,0
395688,2024-05-10 13:00:00+00:00,4757.06,0


In [8]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 395690 entries, 0 to 423017
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         395690 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  395690 non-null  object                 
 2   rides               395690 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(1), object(1)
memory usage: 10.6+ MB


In [9]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [10]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 395690 entries, 0 to 423017
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         395690 non-null  datetime64[us]
 1   pickup_location_id  395690 non-null  object        
 2   rides               395690 non-null  int32         
dtypes: datetime64[us](1), int32(1), object(1)
memory usage: 10.6+ MB


In [11]:
from src.data_utils import transform_ts_data_info_features_and_target
features = transform_ts_data_info_features_and_target(ts_data, window_size=24*28, step_size=24)

Skipping location_id 4757.06: Not enough data to create even one window.
Skipping location_id 4756.04: Not enough data to create even one window.


In [12]:
features, targets = features

In [13]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_hour,pickup_location_id
0,3,19,0,5,7,2,6,2,2,4,...,0,2,0,14,4,0,3,9,2024-04-21 05:00:00,3344.02
1,0,1,0,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,2024-04-12 04:00:00,3779.02
2,1,0,2,7,0,0,3,0,0,1,...,0,0,0,1,4,5,2,1,2024-04-20 15:00:00,3874.01
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2024-05-02 01:00:00,3728.04
4,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,2024-04-19 22:00:00,3121.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
563,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,2024-05-06 20:00:00,3535.03
564,3,0,1,1,2,2,1,1,0,0,...,0,1,0,0,0,0,0,3,2024-04-22 09:00:00,4007.01
565,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2024-04-29 21:00:00,4485.10
566,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2024-05-08 08:00:00,3019.02


In [16]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-05-10 16:49:14,085 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 16:49:14,099 INFO: Initializing external client
2025-05-10 16:49:14,099 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-10 16:49:14,749 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214680
Fetching data from 2024-04-11 20:49:14.085672+00:00 to 2024-05-10 19:49:14.085672+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (12.45s) 
Skipping location_id 4756.04: Not enough data to create even one window.
Skipping location_id 4757.06: Not enough data to create even one window.


In [17]:
features, targets = features

In [18]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_hour,pickup_location_id
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2024-05-09 21:00:00,190 Morgan
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,2024-05-09 21:00:00,2733.03
2,0,1,0,0,0,0,0,0,0,0,...,1,3,1,2,3,1,0,0,2024-05-09 21:00:00,2782.02
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2024-05-09 21:00:00,2821.05
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2024-05-09 21:00:00,2821.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
563,0,1,2,1,1,0,0,0,0,0,...,1,2,1,0,3,3,1,1,2024-05-09 21:00:00,4735.03
564,0,1,0,0,0,0,0,0,0,0,...,2,0,1,0,1,1,0,0,2024-05-09 21:00:00,4735.16
565,0,0,0,0,0,0,0,0,0,1,...,0,0,1,1,1,2,2,0,2024-05-09 21:00:00,4743.04
566,0,0,0,2,0,0,0,0,0,2,...,4,8,7,24,10,6,10,3,2024-05-09 21:00:00,4748.07


In [19]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-05-10 16:51:11,404 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 16:51:11,410 INFO: Initializing external client
2025-05-10 16:51:11,422 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-10 16:51:12,144 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214680


Downloading: 0.000%|          | 0/330679 elapsed<00:00 remaining<?

In [20]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [21]:
predictions

,pickup_location_id,predicted_demand
0,190 Morgan,-0.0
1,2733.03,1.0
2,2782.02,0.0
3,2821.05,0.0
4,2821.06,-0.0
...,...,...
563,4735.03,1.0
564,4735.16,1.0
565,4743.04,2.0
566,4748.07,3.0


In [22]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values

array(['4470.09', '4125.07', '3902.06', '4157.10', '3651.04', '4614.04',
       '4395.07', '4528.01', '4461.04', '4683.02'], dtype=object)